[Source](https://colab.research.google.com/drive/1BiQiw31DT7-cDp1-0ySXvvhzqomTdI-o?usp=sharing&pli=1&authuser=5#scrollTo=_kbS7nRxcMt7)

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190

In [ ]:
from datasets import load_dataset

import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

In [ ]:
dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
model_name = "microsoft/phi-2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
for name, module in model.named_modules():
    print(name)


transformer
transformer.embd
transformer.embd.wte
transformer.embd.drop
transformer.h
transformer.h.0
transformer.h.0.ln
transformer.h.0.resid_dropout
transformer.h.0.mixer
transformer.h.0.mixer.rotary_emb
transformer.h.0.mixer.Wqkv
transformer.h.0.mixer.out_proj
transformer.h.0.mixer.inner_attn
transformer.h.0.mixer.inner_attn.drop
transformer.h.0.mixer.inner_cross_attn
transformer.h.0.mixer.inner_cross_attn.drop
transformer.h.0.mlp
transformer.h.0.mlp.fc1
transformer.h.0.mlp.fc2
transformer.h.0.mlp.act
transformer.h.1
transformer.h.1.ln
transformer.h.1.resid_dropout
transformer.h.1.mixer
transformer.h.1.mixer.rotary_emb
transformer.h.1.mixer.Wqkv
transformer.h.1.mixer.out_proj
transformer.h.1.mixer.inner_attn
transformer.h.1.mixer.inner_attn.drop
transformer.h.1.mixer.inner_cross_attn
transformer.h.1.mixer.inner_cross_attn.drop
transformer.h.1.mlp
transformer.h.1.mlp.fc1
transformer.h.1.mlp.fc2
transformer.h.1.mlp.act
transformer.h.2
transformer.h.2.ln
transformer.h.2.resid_dropout


In [ ]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "Wqkv",
        "out_proj",
        # "inner_attn",
        # "inner_cross_attn",
        "fc1",
        "fc2",
        # "act",
    ]
)

In [ ]:
from transformers import TrainingArguments

new_model = "phi-2-sharded-bf16-custom"
output_dir = "./results"
per_device_train_batch_size = 2
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_steps = 500
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=False,
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 400

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

In [ ]:
for name, module in trainer.model.named_modules():
    print(name)


base_model
base_model.model
base_model.model.transformer
base_model.model.transformer.embd
base_model.model.transformer.embd.wte
base_model.model.transformer.embd.drop
base_model.model.transformer.h
base_model.model.transformer.h.0
base_model.model.transformer.h.0.ln
base_model.model.transformer.h.0.resid_dropout
base_model.model.transformer.h.0.mixer
base_model.model.transformer.h.0.mixer.rotary_emb
base_model.model.transformer.h.0.mixer.Wqkv
base_model.model.transformer.h.0.mixer.Wqkv.base_layer
base_model.model.transformer.h.0.mixer.Wqkv.lora_dropout
base_model.model.transformer.h.0.mixer.Wqkv.lora_dropout.default
base_model.model.transformer.h.0.mixer.Wqkv.lora_A
base_model.model.transformer.h.0.mixer.Wqkv.lora_A.default
base_model.model.transformer.h.0.mixer.Wqkv.lora_B
base_model.model.transformer.h.0.mixer.Wqkv.lora_B.default
base_model.model.transformer.h.0.mixer.Wqkv.lora_embedding_A
base_model.model.transformer.h.0.mixer.Wqkv.lora_embedding_B
base_model.model.transformer.h.0

In [ ]:
for name, module in trainer.model.named_modules():
    if "ln" in name:
        module = module.to(torch.float32)

ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found : LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
ln found :

In [ ]:
trainer.train()

Step,Training Loss
10,1.467700
20,1.498800
30,1.490200
40,1.938100
50,2.156800
60,1.536600
70,1.539200
80,1.673400
90,1.782400
100,2.167400


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=500, training_loss=1.6712945213317871, metrics={'train_runtime': 1261.5912, 'train_samples_per_second': 1.585, 'train_steps_per_second': 0.396, 'total_flos': 1.016813846992896e+16, 'train_loss': 1.6712945213317871, 'epoch': 0.2})

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [ ]:
# Run text generation pipeline with our next model
prompt = "Where is Africa ?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Where is Africa ? [/INST]

<s>Africa is a continent located in the Eastern Hemisphere, mostly in the Southern Hemisphere. It is the second-largest continent in the world, covering about 30% of the Earth's land area. It is home to over 1.2 billion people, and has a diverse range of cultures, languages, and ecosystems. Africa is bordered by the Mediterranean Sea to the north, the Atlantic Ocean to the west, the Indian Ocean to the south, and the Arctic Ocean to the east. It is divided into 54 countries, each with its own history, politics, and economy. Africa is also rich in natural resources, such as oil, gas, diamonds, gold, and minerals. Africa is a continent of contrasts, challenges, and opportunities, and is a vital part of the global community. </s>

<s>Africa is a continent located in the Eastern Hemisphere, mostly in the Southern Hemisphere.


- Copying the Qlora model to gdrive for future use

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r '/content/phi-2-sharded-bf16-custom' "/content/gdrive/MyDrive/ERA1/s27_qlora/llama7b_finetuning"

## Inference from copied model. We will merge the qlora model with original one & save it to gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r "/content/gdrive/MyDrive/ERA1/s27_qlora/llama7b_finetuning/phi-2-sharded-bf16-custom" "/content"

In [ ]:
model_path = "/content/phi-2-sharded-bf16-custom-merged"  # change to your preferred path
model_name = "microsoft/phi-2"
# device_map = {"": 1}

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    trust_remote_code=True
    # device_map=device_map,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import PeftModel
new_model = "phi-2-sharded-bf16-custom"
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('/content/phi-2-sharded-bf16-custom-merged/tokenizer_config.json',
 '/content/phi-2-sharded-bf16-custom-merged/special_tokens_map.json',
 '/content/phi-2-sharded-bf16-custom-merged/vocab.json',
 '/content/phi-2-sharded-bf16-custom-merged/merges.txt',
 '/content/phi-2-sharded-bf16-custom-merged/added_tokens.json',
 '/content/phi-2-sharded-bf16-custom-merged/tokenizer.json')

In [ ]:
!cp -r '/content/phi-2-sharded-bf16-custom-merged' "/content/gdrive/MyDrive/ERA1/s27_qlora/llama7b_finetuning"

## Inference from copied model. We will then download this merged model & perform inference

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r "/content/gdrive/MyDrive/ERA1/s27_qlora/llama7b_finetuning/phi-2-sharded-bf16-custom-merged" "/content"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/phi-2-sharded-bf16-custom-merged"

model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path)

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import pipeline

prompt = "What was the role of indian revolutionaries in indian independence movement ?"  # change to your desired prompt
gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=500)
result = gen(prompt)
print(result[0]['generated_text'])

What was the role of indian revolutionaries in indian independence movement ?

Solution 0:

The Indian revolutionaries were the people who fought against the British colonial rule in India from the late 19th century to the mid-20th century. They used various methods of resistance, such as armed struggle, sabotage, espionage, propaganda, and civil disobedience, to challenge the British authority and demand independence. Some of the most prominent Indian revolutionaries were Bhagat Singh, Sukhdev Thapar, Lala Lajpat Rai, Bhagat Singh, Chandrashekhar Azad, Rash Behari Bose, Subhas Chandra Bose, Jawaharlal Nehru, Mahatma Gandhi, and many others. The Indian revolutionaries played a crucial role in inspiring and mobilizing the masses, raising the nationalist consciousness, and creating a sense of unity and solidarity among the Indians. They also influenced the political and social movements that led to the independence of India in 1947.






Chapter: The Human Eye and Vision

The human eye 

In [ ]:
prompt = "What is a black hole ?"  # change to your desired prompt
gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=200)
result = gen(prompt)
print(result[0]['generated_text'])

What is a black hole ?

A black hole is a region of space where gravity is so strong that nothing, not even light, can escape. Black holes are formed when massive stars collapse at the end of their life cycle. They can have different sizes, from a few kilometers to billions of kilometers in diameter.

What are the properties of a black hole?

A black hole has three main properties: mass, charge, and angular momentum.

Mass: The mass of a black hole determines how much gravity it exerts. The more mass a black hole has, the stronger its gravity is. The mass of a black hole is measured in units of solar masses (M☉), which is the mass of the sun.

Charge: The charge of a black hole is a measure of how much electric charge it has. The charge of a black hole is always zero, because black holes have no electric charge. The charge of a black hole is measured in units


In [ ]:
result

[{'generated_text': 'What is a black hole ?\n\nA black hole is a region of space where gravity is so strong that nothing, not even light, can escape. Black holes are formed when massive stars collapse at the end of their life cycle. They can have different sizes, from a few kilometers to billions of kilometers in diameter.\n\nWhat are the properties of a black hole?\n\nA black hole has three main properties: mass, charge, and angular momentum.\n\nMass: The mass of a black hole determines how much gravity it exerts. The more mass a black hole has, the stronger its gravity is. The mass of a black hole is measured in units of solar masses (M☉), which is the mass of the sun.\n\nCharge: The charge of a black hole is a measure of how much electric charge it has. The charge of a black hole is always zero, because black holes have no electric charge. The charge of a black hole is measured in units'}]

## Inferencing in CPU - Using the custom model from HF...model = AutoModelForCausalLM.from_pretrained("anilbhatt1/phi2-oasst-guanaco-bf16-custom")

In [2]:
!pip install -q transformers einops

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "anilbhatt1/phi2-oasst-guanaco-bf16-custom"

model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path)

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import pipeline

prompt = "What is a black hole ?"  # change to your desired prompt
gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=250)
result = gen(prompt)
print(result[0]['generated_text'])

## Gradio inferencing in cpu (high ram)

In [1]:
!pip install -q gradio==3.50.2
!pip install -q transformers einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account 

In [2]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_path = "anilbhatt1/phi2-oasst-guanaco-bf16-custom"
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path)

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def generate_text(prompt, response_length):

    prompt = str(prompt)
    max_len = int(response_length)

    gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_len)
    result = gen(prompt)
    output_msg = result[0]['generated_text']
    return output_msg

def gradio_fn(prompt, response_length):
    output_txt_msg = generate_text(prompt, response_length)
    return output_txt_msg

markdown_description = """
- This is a Gradio app that answers the query you ask it
- Uses **microsoft/phi-2 qlora** optimized model finetuned on **timdettmers/openassistant-guanaco** dataset
"""
demo = gr.Interface(fn=gradio_fn,
                    inputs=[gr.Textbox(info="How may I help you ? please enter your prompt here..."),
                            gr.Slider(value=50, minimum=50, maximum=200, \
                                      info="Choose a response length min chars=50, max=200")],
                    outputs=gr.Textbox(),
                    title="phi2 - Dialog Partner",
                    description=markdown_description,
                    article=" **Credits** : https://github.com/mshumer/gpt-llm-trainer ")

In [ ]:
demo.queue().launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://407263821dce49b9cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
